MAX

In [2]:
import pandas as pd
import sklearn as sk 

In [23]:
transactions = pd.read_csv("C:\codding\Data_Hack\Sber hack\\transactions.csv")
transactions

,client_id,trans_time,mcc_code,trans_type,amount,term_id,trans_city
0,d1bbbc9a0e0410d3cf12a3d2f44f3450,35 08:24:41,4829,2370,-1808.56,NaN,Tver
1,81c7349f7bdcb661bc7d5f5f0fdac6aa,351 13:27:15,6011,7010,7954.51,442207,Kazan
2,fc0c7c937f139bd6d1c5b5d0a0023dd3,150 16:03:59,6011,2010,-12293.56,NaN,Khabarovsk
3,3495487d5beaa5caaa0682fddc38379a,373 11:54:12,6010,7070,49.52,888987,Kaliningrad
4,07e279042b1da23710fa2f65034d30b9,340 13:11:45,6010,7070,45.29,888918,Saint Petersburg
...,...,...,...,...,...,...,...
3563524,e91858185ae43cf3970220748893a01b,328 13:45:38,6010,7070,227.40,888907,Penza
3563525,429b6c8255f13bdb506fdbd6e342ccb1,10 19:22:35,4814,1030,-1084.44,NaN,Khabarovsk
3563526,9443b6a0d948cf368d8d72eb295bdfdf,167 23:59:15,5411,1110,-569.20,NaN,Tver
3563527,1be75d38ad89c0f2b4d85b95f0fd8069,101 23:59:03,5814,1110,-71.43,NaN,Vladivostok


In [15]:
mcc_unique_num = transactions["mcc_code"].nunique()
trans_t_unique_num = transactions["trans_type"].nunique()

print(f"Number of unique mcc codes is {mcc_unique_num}, number of unique trans_t codes is {trans_t_unique_num}")

Number of unique mcc codes is 184, number of unique trans_t codes is 73


In [5]:
city_unique_num = transactions["trans_city"].nunique()

print(f"Number of unique cities is {city_unique_num}")

Number of unique cities is 10


In [21]:
id_unique_num = transactions["client_id"].nunique()

print(f"Number of unique IDs is {id_unique_num}")

Number of unique IDs is 8384


In [6]:
train = pd.read_csv("C:\codding\Data_Hack\Sber hack\hse_sber_hack\\train.csv")

In [7]:
transactions_merged = transactions.merge(right=train)

In [16]:
transactions_merged.head(100)

,client_id,trans_time,mcc_code,trans_type,amount,term_id,trans_city,Unnamed: 0,gender
0,d1bbbc9a0e0410d3cf12a3d2f44f3450,35 08:24:41,4829,2370,-1808.56,NaN,Tver,7196,0
1,d1bbbc9a0e0410d3cf12a3d2f44f3450,105 12:57:32,4829,2370,-3390.41,NaN,Tver,7196,0
2,d1bbbc9a0e0410d3cf12a3d2f44f3450,455 19:32:01,4814,1030,-144.50,889003,Tver,7196,0
3,d1bbbc9a0e0410d3cf12a3d2f44f3450,83 09:22:26,6011,2010,-3542.30,NaN,Tver,7196,0
4,d1bbbc9a0e0410d3cf12a3d2f44f3450,74 13:31:57,6011,2010,-3542.70,NaN,Tver,7196,0
...,...,...,...,...,...,...,...,...,...
95,d1bbbc9a0e0410d3cf12a3d2f44f3450,154 14:50:37,5814,1010,-287.67,NaN,Tver,7196,0
96,d1bbbc9a0e0410d3cf12a3d2f44f3450,143 11:03:47,5211,1110,-378.97,NaN,Tver,7196,0
97,d1bbbc9a0e0410d3cf12a3d2f44f3450,4 13:07:51,4829,2370,-2833.84,NaN,Tver,7196,0
98,d1bbbc9a0e0410d3cf12a3d2f44f3450,374 12:43:05,6011,2010,-434.58,954125,Tver,7196,0


In [17]:
transactions_merged['client_id'].nunique()

7560

In [18]:
split_date = transactions_merged["trans_time"].str.split(' ')
split_date.head()

0     [35, 08:24:41]
1    [105, 12:57:32]
2    [455, 19:32:01]
3     [83, 09:22:26]
4     [74, 13:31:57]
Name: trans_time, dtype: object

In [19]:
somename = pd.read_csv("C:\codding\Data_Hack\Sber hack\hse_sber_hack\\test.csv")